<h1>Toronto neighbourhood segmentation<h1>

The current script is made to scrape the necessary data for the segmentation task from a designated wikipedia page, data will then be used to segment neighborhoods in Toronto, Canada.
Following that, the data will be used to explore and segment and cluster these neighbourhood

Importing necessary libraries:

In [2]:
#to grab the html file: urlopen
from urllib.request import urlopen as uReq
#to parse the html file: beautiful soup 
from bs4 import BeautifulSoup as soup 
#to manage and manipulate the data into dataframes: pandas
import pandas as pd 

<h4> Scraping and cleaning the data </h4>

In [3]:
#the wikipedia url to scrape the data from
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#dowloading the webpage and stroing it 
uClient = uReq(url)
page_html = uClient.read()

In [5]:
#parsing the html page 
page_soup= soup(page_html, 'html.parser')
page_soup.h1

<h1 class="firstHeading" id="firstHeading" lang="en">List of postal codes of Canada: M</h1>

In [6]:
# The first tr contains the field names.
headings = ['Postcode', 'Borough', 'Neighbourhood']
headings

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
# creating a dataframe with the data in the table after extracting it 
datasets = []
for row in page_soup.find_all("tr")[1:]:
    dataset =  (td.get_text() for td in row.find_all("td"))
    datasets.append(dataset)
df = pd.DataFrame(datasets )
# initial cleaning, assigning headers and removing empty "none" data 
labels= range(3,31)
none = range (287,292)
df.drop(labels, inplace = True, axis= 1)
df.drop(none, inplace = True, axis= 0)
df.columns=headings
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
...,...,...,...
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n
285,M8Z,Etobicoke,South of Bloor\n


In [8]:
# cleaning the data 
df=df[df.Borough != 'Not assigned']
# removing the "\n" that used to signal the end of a line
df['Neighbourhood']= df['Neighbourhood'].apply(lambda x: str(x)[:-1])
# assigning the missing neighborhoods the Borough
df[df.Neighbourhood == 'Not assigned\n'] = df[df.Neighbourhood == 'Not assigned\n'].Borough
df.index= range(210)
df

C:\Users\Skrzydelka\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Skrzydelka\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Skrzydelka\Anaconda3\lib\site-packages\pandas\core\frame.py:3488: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [9]:
# combining  multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.head(20)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


<h4> Getting geospacial data and creating map </h4>

In [12]:
#add Geo-spatial data
dfll= pd.read_csv("Geospatial_Coordinates.csv")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [ ]:
address = 'New York City, NY'
# defining agent 
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))